In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

In [2]:
user_content = pd.read_csv("data/azure/VAS_user_content.csv")

In [ ]:
vas_scored_df

### Applying Business rules

In [3]:
user_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27555 entries, 0 to 27554
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   event_source.hash                    27555 non-null  object 
 1   Location_Identifier                  3078 non-null   object 
 2   Sub_Type                             27555 non-null  object 
 3   Sub_Usage                            27555 non-null  object 
 4   GENDER                               4104 non-null   object 
 5   AGE                                  4104 non-null   float64
 6   Sub_Update_Status                    19648 non-null  object 
 7   Sub_Update                           19648 non-null  object 
 8   Having_Insurance                     27555 non-null  object 
 9   Having_Senior_Citizen_Scaled         27555 non-null  float64
 10  Having_Kids_at_Home_Scaled           27555 non-null  float64
 11  Having_Teenagers_Students_Sc

In [ ]:
vas_df = pd.merge(user_content[["event_source.hash","Sub_Type"]], vas_scored_df, on=['event_source.hash'] how='left', indicator='Exist')